In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import os

In [ ]:
# Stacy's code starts here

In [ ]:
# Load naturalization table
nat_file = 'fy2018_naturalization.xlsx'
filepath = os.path.join('.', 'Resources', nat_file)

naturalization = pd.read_excel(filepath, header=5)

In [ ]:
# Drop notes data at end of document
naturalization.drop(labels=range(112,118), inplace=True)
naturalization.tail()

In [ ]:
# Rename columns
naturalization.rename(columns={
    'filed': 'Petitions filed',
    'Total': 'Naturalized, total',
    'denied': 'Petitions denied'
}, inplace=True)

# Drop unneeded columns
naturalization = naturalization.drop(labels=['Civilian', 'Military 2', 'Not reported'], axis='columns')

In [ ]:
# Find funny/footnoted years and fix them
for index, row in naturalization.iterrows():
    
    year = naturalization.loc[index, 'Year']
    
    if len(str(year)) > 4:
        year = int(year[0:5])
        naturalization.loc[index, 'Year'] = year
    else:
        pass

# Set year as index
naturalization.set_index(keys=['Year'], inplace=True)

In [ ]:
# Change datatype to int for all columns
for c in naturalization.columns:
    naturalization[c] = naturalization[c].astype('int')

In [ ]:
naturalization.head()

In [ ]:
# Plot the data
plt.plot(naturalization.index, naturalization[['Petitions filed', 'Naturalized, total', 'Petitions denied']])
plt.tight_layout()

# Spike in 1996 - and general increase - can be explained by this bill: https://www.vox.com/2016/4/28/11515132/iirira-clinton-immigration






# Need to plot just the 2000 - 2010, then 2015 - 2018 data - to examine whether there are trends

In [ ]:
# Stacy's code ends here

In [ ]:
# Kana's code starts here

In [ ]:
# Kana's code ends here

In [ ]:
# Satish's code starts here

In [ ]:
# Satish's code ends here

In [ ]:
# Umar's code starts here

In [ ]:
# Umar's code ends here